<a href="https://colab.research.google.com/github/TheCaveOfAdullam/study3/blob/main/1017Test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torch_pruning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 2.9 MB/s eta 0:00:00


In [3]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils as utils
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder
import torch.nn.utils.prune as prune
import torch_pruning as tp

In [4]:
# 기본 경로 설정
base_dir = '/content/drive/MyDrive/ship_motor10'
categories = ['normal', 'fault_BB', 'fault_RI', 'fault_SM']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 데이터 로드 및 전처리 함수 정의
class VibrationDataset(Dataset):
    def __init__(self, base_dir, split, categories, transform=None):
        self.X = []
        self.y = []
        self.transform = transform
        split_dir = os.path.join(base_dir, split)
        for category in categories:
            category_dir = os.path.join(split_dir, category)
            files = os.listdir(category_dir)
            for file in files:
                file_path = os.path.join(category_dir, file)
                data = pd.read_csv(file_path, header=None).values
                data = pd.to_numeric(data.flatten(), errors='coerce').reshape(-1, data.shape[1])
                data = np.nan_to_num(data).astype('float32')  # NaN 값을 0으로 대체하고, float32로 변환
                self.X.append(data)
                self.y.append(category)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X = self.X[idx]
        y = self.y[idx]
        X = X.reshape(-1)  # 마지막 차원을 시퀀스에 병합하여 [sequence_length]로 변환
        return torch.tensor(X, dtype=torch.float32), y

# 레이블 인코딩
label_encoder = LabelEncoder()

# 데이터셋 준비
train_dataset = VibrationDataset(base_dir, 'train', categories)
val_dataset = VibrationDataset(base_dir, 'validation', categories)
test_dataset = VibrationDataset(base_dir, 'test', categories)

# 레이블 인코딩 및 원-핫 인코딩
y_train_encoded = label_encoder.fit_transform([y for _, y in train_dataset])
y_val_encoded = label_encoder.transform([y for _, y in val_dataset])
y_test_encoded = label_encoder.transform([y for _, y in test_dataset])

# 데이터셋에 레이블 추가
train_dataset.y = y_train_encoded
val_dataset.y = y_val_encoded
test_dataset.y = y_test_encoded

# 데이터 로더
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=16, stride=16)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3, stride=1)
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, stride=1)
        self.conv4 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, stride=1)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)

        conv1_output_size = (24002 - 16) // 16 + 1
        pool1_output_size = conv1_output_size // 2
        conv2_output_size = (pool1_output_size - 3) // 1 + 1
        conv3_output_size = (conv2_output_size - 5) // 1 + 1
        conv4_output_size = (conv3_output_size - 5) // 1 + 1
        pool2_output_size = conv4_output_size // 2

        self.fc1 = nn.Linear(128 * pool2_output_size, 5000)
        self.fc2 = nn.Linear(5000, 1000)
        self.fc3 = nn.Linear(1000, len(categories))

    def forward(self, x):
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = torch.relu(x)
        x = self.conv3(x)
        x = torch.relu(x)
        x = self.conv4(x)
        x = torch.relu(x)
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
# 1차 테일러 전개 기반 비구조적 프루닝
def prune_by_taylor(model, threshold=0.01):
    for name, module in model.named_modules():
        # Conv1d 레이어와 FC (Linear) 레이어 모두에 대해 적용
        if isinstance(module, nn.Conv1d) or isinstance(module, nn.Linear):
            if module.weight.grad is None:
                raise ValueError(f"Gradients not found for {name}. Run backward pass before pruning.")

            # 중요도 계산 (Taylor 기반)
            importance = torch.abs(module.weight * module.weight.grad)
            mask = importance > threshold
            with torch.no_grad():
                module.weight[~mask] = 0  # 중요도가 낮은 가중치 0으로 설정
    print(f"Taylor expansion-based pruning with threshold: {threshold} applied.")

# 0 비율 기반 필터 감지 및 구조적 프루닝 적용 함수
def detect_and_apply_structural_pruning_with_zero_ratio(model, prune_threshold=0.7, example_inputs=None):
    if example_inputs is None:
        example_inputs = torch.randn(1, 1, 24002).to(next(model.parameters()).device)

    # Dependency Graph 생성 (의존성 추적)
    DG = tp.DependencyGraph().build_dependency(model, example_inputs=example_inputs)

    total_pruned = 0
    # Conv 및 FC 레이어 프루닝
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv1d) or isinstance(module, nn.Linear):
            # 각 필터 또는 뉴런에서 0 비율 계산
            weight_data = module.weight.detach().cpu().numpy()
            filter_zero_percentage = np.mean(weight_data == 0, axis=(1, 2) if isinstance(module, nn.Conv1d) else 1)
            prune_indices = np.where(filter_zero_percentage >= prune_threshold)[0]

            # 프루닝 대상 필터 또는 뉴런 제거
            if len(prune_indices) > 0 and len(prune_indices) < module.weight.shape[0]:
                pruning_group = None
                if isinstance(module, nn.Conv1d):
                    pruning_group = DG.get_pruning_group(module, tp.prune_conv_out_channels, idxs=prune_indices)
                elif isinstance(module, nn.Linear):
                    pruning_group = DG.get_pruning_group(module, tp.prune_linear_out_channels, idxs=prune_indices)

                if pruning_group is not None:
                    pruning_group.prune()  # 의존성 그래프를 통해 다음 레이어와의 연결을 조정
                    total_pruned += len(prune_indices)
                    print(f"Pruned {len(prune_indices)} filters/neuron(s) from {name}.")
                else:
                    print(f"Skipping pruning for {name} as no pruning group was generated.")
            else:
                print(f"Skipping pruning for {name} as it would remove all filters/neuron(s).")

    print(f"Structural pruning based on zero ratio applied. {total_pruned} filters/neuron(s) pruned in total.")
    return model

In [7]:
# 모델 학습 함수
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=5, max_norm=1.0):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.unsqueeze(1).to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()

            # 그래디언트 클리핑
            utils.clip_grad_norm_(model.parameters(), max_norm)
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_loss, val_accuracy = evaluate_model(model, val_loader, criterion)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/total:.4f}, Accuracy: {100 * correct/total:.2f}%, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    return model

# 모델 평가 함수
def evaluate_model(model, loader, criterion):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.unsqueeze(1).to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return val_loss / total, 100 * correct / total

In [8]:
# 프루닝 과정
def prune_and_retrain(model, train_loader, val_loader, test_loader, criterion, device, optimizer_params, threshold_taylor=0.01, prune_threshold=0.7):
    # 옵티마이저 설정
    optimizer = optim.Adam(model.parameters(), **optimizer_params)

    # 1차적으로 모델 학습
    print("Initial training before pruning")
    model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=5)

    # 1차 테일러 전개 기반 비구조적 프루닝
    print("Step 1: Taylor expansion-based pruning")
    for inputs, labels in train_loader:
        inputs, labels = inputs.unsqueeze(1).to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()  # 경사도 계산
        break  # 경사도만 계산하므로 한 배치만 사용
    prune_by_taylor(model, threshold_taylor)

    # 재학습
    optimizer = optim.Adam(model.parameters(), **optimizer_params)
    model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=5)

    # 구조적 프루닝 (0 비율이 70% 이상인 필터와 뉴런 제거)
    print("Step 2: Structural pruning based on zero ratio")
    model = detect_and_apply_structural_pruning_with_zero_ratio(model, prune_threshold=prune_threshold)

    # 다시 재학습
    optimizer = optim.Adam(model.parameters(), **optimizer_params)
    model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=5)

    # 최종 테스트 평가
    print("Final evaluation on the test set...")
    test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
    print(f'Final Test Loss: {test_loss:.4f}, Final Test Accuracy: {test_accuracy:.2f}%')

    return model

In [9]:
# 파라미터 설정 및 프루닝 실행
model = CNNModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer_params = {'lr': 0.0001, 'weight_decay': 1e-5}

# 프루닝 및 재학습 실행
model = prune_and_retrain(model, train_loader, val_loader, test_loader, criterion, device, optimizer_params, threshold_taylor=0.0001, prune_threshold=0.7)

Initial training before pruning
Epoch [1/5], Loss: 0.0391, Accuracy: 49.94%, Val Loss: 0.0393, Val Accuracy: 50.00%
Epoch [2/5], Loss: 0.0390, Accuracy: 50.00%, Val Loss: 0.0392, Val Accuracy: 50.00%
Epoch [3/5], Loss: 0.0333, Accuracy: 56.83%, Val Loss: 0.0185, Val Accuracy: 80.70%
Epoch [4/5], Loss: 0.0146, Accuracy: 79.75%, Val Loss: 0.0109, Val Accuracy: 92.44%
Epoch [5/5], Loss: 0.0083, Accuracy: 88.97%, Val Loss: 0.0401, Val Accuracy: 33.44%
Step 1: Taylor expansion-based pruning
Taylor expansion-based pruning with threshold: 0.0001 applied.
Epoch [1/5], Loss: 0.0180, Accuracy: 82.40%, Val Loss: 0.0040, Val Accuracy: 97.11%
Epoch [2/5], Loss: 0.0050, Accuracy: 93.87%, Val Loss: 0.0035, Val Accuracy: 95.44%
Epoch [3/5], Loss: 0.0046, Accuracy: 94.33%, Val Loss: 0.0017, Val Accuracy: 98.41%
Epoch [4/5], Loss: 0.0082, Accuracy: 90.22%, Val Loss: 0.0354, Val Accuracy: 70.63%
Epoch [5/5], Loss: 0.0054, Accuracy: 93.38%, Val Loss: 0.0013, Val Accuracy: 98.26%
Step 2: Structural pruning

In [10]:
# 프루닝 및 재학습 후 모델 평가
print("Final evaluation on the test set...")
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
print(f'Final Test Loss: {test_loss:.4f}, Final Test Accuracy: {test_accuracy:.2f}%')

Final evaluation on the test set...
Final Test Loss: 0.0001, Final Test Accuracy: 99.89%
